# Chapter 4. Forms and View Logic

## 4.1 Introduction

### 4.1.1 Forms

(1) Extension: Flask-WTF

(2) Form templates

(3) Validating input

(4) Handling logic

### 4.1.2 Views

(1) POST method

(2) Redirection

(3) Messaging flashing

## 4.2 Demo: Views and forms

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for

from logging import DEBUG

app = Flask(__name__)
app.logger.setLevel(DEBUG)

bookmarks = []

def store_bookmark(url):
    bookmarks.append(dict(
        url = url,
        user = "reindert",
        date = datetime.utcnow()    
    ))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    if request.method == 'POST':
        url = request.form['url']
        store_bookmark(url)
        app.logger.debug('stored url: ' + url)
        return redirect(url_for('index'))
    return render_template('add.html')
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 4.3 Review: Views, forms, post-redirect-get

### 4.3.1 Post-Redirect-Get

    Form page view (GET)   
==> Submit (POST)  
==> Validation error? == Yes ==> `render_template()` shows the error message     
                      == No  ==> `redirect()`
                      
### 4.3.2 Views and forms

(1) `@app.route('/add', methods=['GET', 'POST'])`

(2) `if request.method == 'POST':`

(3) `url = request.form['url']`

### 4.3.3 The request object

(1) Globally available

* But temporarily bound to the current request

* Don't use it when no request is active (outside view functions)

(2) Some attributes of the request object:

* form
* args
* cookies
* headers
* files
* method

## 4.4 Session and Flashes

### 4.4.1 Session object

(1) Remember data between requests

(2) Works by setting a cookie

(3) Data associated with the user's **HTTP Session**

(4) Is a Flask context global, like `request`

### 4.4.2 Using the session

(1) Need to set `Flask.secret_key` for creating cookies

(2) Store values in it like a dict

### 4.4.3 Flashing messages

(1) Use `flash()`

(2) Available in template throught `get_flashed_messages()`

In [1]:
# Get the Flask.secret_key

import os
os.urandom(24)

b'J\xcb\xb3\xb8\xa0nK\x0e\x8a\xcd\x08\xc6=\xeb$\xc3&Zx\x8dT\x8a\x92E'

In [2]:
import secrets
secrets.token_bytes(24)

b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for, flash

app = Flask(__name__)

bookmarks = []
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'

def store_bookmark(url):
    bookmarks.append(dict(
        url = url,
        user = "reindert",
        date = datetime.utcnow()    
    ))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    if request.method == 'POST':
        url = request.form['url']
        store_bookmark(url)
        flash("Stored bookmark '{}'".format(url))
        return redirect(url_for('index'))
    return render_template('add.html')
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 4.5 If and For

### 4.5.1 If

```python
{% if expression %} ... {% else %} ... {% endif %}
```

`{% elfi %}`

### 4.5.2 For

Jinja provides a loop variable inside `for`.

```python
{% for var in expression %} ... {% endfor %}
```

### 4.5.3 with

```python
{% with var = expression %} ... {% endwith %}
```

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for, flash

app = Flask(__name__)

bookmarks = []
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'

def store_bookmark(url):
    bookmarks.append(dict(
        url = url,
        user = "reindert",
        date = datetime.utcnow()    
    ))
    
def new_bookmarks(num):
    return sorted(bookmarks, key=lambda bm: bm['date'], reverse=True)[:num]

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=new_bookmarks(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    if request.method == 'POST':
        url = request.form['url']
        store_bookmark(url)
        flash("Stored bookmark '{}'".format(url))
        return redirect(url_for('index'))
    return render_template('add.html')
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 4.6 Demo: Using WTForms

### 4.6.1 Flask WTF

(1) Flask extension

(2) WTForms integration

(3) Render forms

(4) Validate forms

### 4.6.2 How to use Flask WTF

```bash
$ pip install flask-wtf
```

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, url

class BookmarkForm(Form):
    url = URLField('url', validators=[DataRequired(), url()])
    description = StringField('description')

In [ ]:
# SAVE AS thermos.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for, flash

from forms import BookmarkForm

app = Flask(__name__)

bookmarks = []
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'

def store_bookmark(url, description):
    bookmarks.append(dict(
        url = url,
        description = description,
        user = "reindert",
        date = datetime.utcnow()    
    ))
    
def new_bookmarks(num):
    return sorted(bookmarks, key=lambda bm: bm['date'], reverse=True)[:num]

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=new_bookmarks(5))
                           
@app.route('/add', methods=['GET', 'POST'])
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        store_bookmark(url, description)
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('add.html', form=form)
    
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  
  
if __name__ == '__main__':
    app.run(debug=True)

## 4.7 Demo: Showing form errros

Modify `add.html` to show the form errors.

## 4.8 Review: Using WTForms

### 4.8.1 Flask-WTF Form Classes

(1) Simple Python class inheriting from `flask_wtf.Form`.

(2) Fields

* Many field classes for different kinds of input
* `StringField`, `DateField`, `BooleanField`, `SelectField`, `HiddenField`, etc.
* Note: `flask_wtf` includes HTML5 forms in `flask.ext.wtf.html5`

(3) Pass a list of validators to the field

```python
url = URLField('url', validators=[DataRequired(), url()])
```

### 4.8.2 WTForms in views

(1) Create a `form` instance

* `form = BookmarkForm()`

* This will be filled with any data from the request, if any

(2) Check: was the form submitted and does it validate?

`form.validate_on_submit()`

(3) Get data from fields

`url = form.url.data`

(4) Make sure to pass the form to the template

`render_template('add.html', form=form)`

### 4.8.3 WTForms in templates

(1) Don't forget CSRF protection!

* `{{ form.hidden_tag() }}`
* Need a secret key, i.e. `Flask.secret_key`

(2) Render fields

* `{{ form.url }}`
* Keyword arguments will become HTML attributes
* `{{ form.url(class="fancy") }}`

(3) Errors

`{% for error in form.url.errors %}`

## 4.9 Custom validation

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, url

class BookmarkForm(Form):
    url = URLField('url', validators=[DataRequired(), url()])
    description = StringField('description')
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        return True

## 4.10 Demo: A Jinja macro for rendering forms

(1) Define the Jinja macro.

```html
<!-- SAVE AS form_macros.html -->

{% macro render_field(field) %}
    <tr {% if field.errors %} class="error" {% endif %}>
        <td>{{ field.label }} </td> 
        <td>{{ field(**kwargs)|safe }}</td>
    </tr>
    <tr class="error">
        <td></td>
        <td>
            <ul>
                {% for error in field.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
            </ul>
        </td>
    </tr>
{% endmacro %}
```

(2) Use the Jinja macro in the template.

```html
<!-- SAVE as add.html -->
```

{% extends "base.html" %}
{% from "form_macros.html" import render_field %}

{% block title %}
Pytwis -- Add a URL
{% endblock %}

{% block content %}
    <section>
        <h1>Add a new URL</h1>
        <form action="" method="post">
            {{ form.hidden_tag() }}
            <table>
                {{ render_field(form.url, size=50) }}
                {{ render_field(form.description, size=50) }}
                
                <tr>
                    <td></td>
                    <td><button type="submit">Submit</button></td>
                </tr>
            </table>
        </form>
    </section>
{% endblock %}

{% block sidebar %}
{% endblock %}

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, url

class BookmarkForm(Form):
    url = URLField('The URL for your bookmark', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:')
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        return True

## 4.11 Resources and summary

### 4.11.1 Resources

(1) List of control structures in Jinja

http://jinja.pocoo.org/docs/dev/templates/#list-of-control-structures

(2) Flask WTForm integration

https://flask-wtf.readthedocs.org

(3) WTForm documentation

http://wtforms.readthedocs.org/en/latest/index.html

(4) Post-Redirect-Get wiki

https://en.wikipedia.org/wiki/Post/Redirect/Get

### 4.11.2 Summary

